In [2]:
%matplotlib inline

#System
import os
import shutil

#Base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt2
import cv2
import random

#Keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPool2D, UpSampling2D
from keras.layers import Dropout,BatchNormalization, Concatenate
from keras.optimizers import Adam, SGD, RMSprop

#Preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence, to_categorical

#Models
from keras.applications.vgg16 import VGG16

#GPU
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto() 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config) 
K.set_session(sess)

#Seed
seed = 99
np.random.seed(seed)
random.seed(seed)

## Генераторы данных

In [3]:
img_shape = 224
batch_size = 2

train_size = len(os.listdir(path="data/ADE20K_formated/train/img/img/"))
val_size = len(os.listdir(path="data/ADE20K_formated/val/img/img/"))

train_dir = "data/ADE20K_formated/train/"
val_dir = "data/ADE20K_formated/val/"

In [4]:
def data_gen(data_dir, batch_size):
    img_folder = data_dir + "img/img/"
    mask_folder = data_dir + "mask/mask/"
    
    n = os.listdir(img_folder)
    random.shuffle(n)
    
    for i in range(len(n)):
        n[i] = n[i].split(".")[0]
        
    c = 0
    while (True):
        img = np.zeros((batch_size, img_shape, img_shape, 3)).astype('float')
        mask = np.zeros((batch_size, img_shape, img_shape, 253)).astype("uint8")

        for i in range(c, c + batch_size):  

            train_img = cv2.imread(img_folder + '/' + n[i] + ".jpg") / 255.
            train_img =  cv2.resize(train_img, (img_shape, img_shape))

            img[i - c] = train_img 

            train_mask = cv2.imread(mask_folder + '/' + n[i] + ".png", cv2.IMREAD_GRAYSCALE)
            train_mask = cv2.resize(train_mask, (img_shape, img_shape), interpolation = cv2.INTER_NEAREST)
            train_mask = train_mask.reshape(img_shape, img_shape, 1)
            train_mask = to_categorical(train_mask, num_classes=253)
            
            mask[i - c] = train_mask

        c += batch_size
        
        if (c + batch_size >= len(os.listdir(img_folder))):
            c = 0
            random.shuffle(n)
                  
        yield img, mask

In [5]:
train_gen = data_gen(train_dir, batch_size=batch_size)
val_gen = data_gen(val_dir, batch_size=batch_size)

In [6]:
block0_input = Input(shape=(img_shape, img_shape, 3))

block1_conv1 = Conv2D(64, (3, 3), padding="same", activation="relu")(block0_input)
block1_conv2 = Conv2D(64, (3, 3), padding="same", activation="relu")(block1_conv1)
block1_conv3 = Conv2D(64, (3, 3), padding="same", activation="relu")(block1_conv2)
block1_pool1 = MaxPool2D(2)(block1_conv3)

block2_conv1 = Conv2D(128, (3, 3), padding="same", activation="relu")(block1_pool1)
block2_conv2 = Conv2D(128, (3, 3), padding="same", activation="relu")(block2_conv1)
block2_conv3 = Conv2D(128, (3, 3), padding="same", activation="relu")(block2_conv2)
block2_pool1 = MaxPool2D(2)(block2_conv3)

block3_conv1 = Conv2D(256, (3, 3), padding="same", activation="relu")(block2_pool1)
block3_conv2 = Conv2D(256, (3, 3), padding="same", activation="relu")(block3_conv1)
block3_conv3 = Conv2D(256, (3, 3), padding="same", activation="relu")(block3_conv2)
block3_pool1 = MaxPool2D(2)(block3_conv3)

block4_conv1 = Conv2D(512, (3, 3), padding="same", activation="relu")(block3_pool1)
block4_conv2 = Conv2D(512, (3, 3), padding="same", activation="relu")(block4_conv1)
block4_conv3 = Conv2D(512, (3, 3), padding="same", activation="relu")(block4_conv2)
block4_upsa1 = UpSampling2D(2, interpolation="bilinear")(block4_conv3)

block5_conc1 = Concatenate()([block3_conv3, block4_upsa1])
block5_conv1 = Conv2D(256, (3, 3), padding="same", activation="relu")(block5_conc1)
block5_conv2 = Conv2D(256, (3, 3), padding="same", activation="relu")(block5_conv1)
block5_conv3 = Conv2D(256, (3, 3), padding="same", activation="relu")(block5_conv2)
block5_upsa1 = UpSampling2D(2, interpolation="bilinear")(block5_conv3)

block6_conc1 = Concatenate()([block2_conv3, block5_upsa1])
block6_conv1 = Conv2D(128, (3, 3), padding="same", activation="relu")(block6_conc1)
block6_conv2 = Conv2D(128, (3, 3), padding="same", activation="relu")(block6_conv1)
block6_conv3 = Conv2D(128, (3, 3), padding="same", activation="relu")(block6_conv2)
block6_upsa1 = UpSampling2D(2, interpolation="bilinear")(block6_conv3)

block7_conc1 = Concatenate()([block1_conv3, block6_upsa1])
block7_conv1 = Conv2D(64, (3, 3), padding="same", activation="relu")(block7_conc1)
block7_conv2 = Conv2D(64, (3, 3), padding="same", activation="relu")(block7_conv1)
block7_conv3 = Conv2D(64, (3, 3), padding="same", activation="relu")(block7_conv2)

block8_output = Conv2D(253, (1, 1), padding="same", activation="softmax")(block7_conv3)

model = Model(inputs=block0_input, outputs=block8_output)

In [7]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 224, 224, 64) 36928       conv2d_2[0][0]                   
____________________________________________________________________________________________

In [8]:
model.compile(Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=["accuracy"])

In [9]:
 %%time
history = model.fit_generator(train_gen, 
                              epochs=50, 
                              steps_per_epoch=500,
                              validation_data=val_gen, 
                              validation_steps=50,
                              verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
500/500 [==============================] - 303s 607ms/step - loss: 2.8935 - accuracy: 0.4986 - val_loss: 3.1505 - val_accuracy: 0.5024
Epoch 2/50
500/500 [==============================] - 298s 596ms/step - loss: 2.6477 - accuracy: 0.5098 - val_loss: 1.9423 - val_accuracy: 0.5155
Epoch 3/50
500/500 [==============================] - 297s 594ms/step - loss: 2.6488 - accuracy: 0.5089 - val_loss: 2.3770 - val_accuracy: 0.4911
Epoch 4/50
469/500 [===========================>..] - ETA: 17s - loss: 2.6791 - accuracy: 0.4974

KeyboardInterrupt: 